# Analysis of Higher Dimensional (Spatial) Data

### Research question:

**On global scale earth surface temperatures are rising significantely. What about Potsdam, Germany?**

***

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)

***

## Global Mean Temperatur Data

Source: [Berkeley Earth](http://berkeleyearth.org/data/)

In [ ]:
## Load data from the net
#URL = "http://berkeleyearth.lbl.gov/auto/Global/Complete_TAVG_complete.txt"
#df_glob = pd.read_csv(URL, skiprows=33, delim_whitespace=True)

## Alternative: load from disk
DATA = "../data/Complete_TAVG_complete.txt"
df_glob = pd.read_csv(DATA, skiprows=33, delim_whitespace=True)
df_glob.head()

In [ ]:
df_glob = df_glob.iloc[:,[0,1,2,3]]
df_glob.columns = ['year', 'month', 'anomaly', 'uncertainty']
df_glob.sample(10)

In [ ]:
df_glob.index = pd.to_datetime(df_glob['year'].astype(str) + '/' + df_glob['month'].astype(str), 
                               format="%Y/%m").dt.to_period('M')
df_glob.sample(10)

In [ ]:
df_glob.anomaly.plot(color='k', label="temperature anomaly")
w=12*10 # 10-year window 
df_glob_anomaly = df_glob.anomaly.rolling(window=w, center=True).mean()
df_glob_anomaly.plot(color="r", label='10-year mean')
plt.fill_between(x=df_glob.index, 
                 y1=(df_glob.anomaly + df_glob.uncertainty).rolling(window=w, center=True).mean(),
                 y2=(df_glob.anomaly - df_glob.uncertainty).rolling(window=w, center=True).mean(),
                color="gray", label='uncertainty')
plt.ylabel('Temperature anomaly [°C]')
plt.title("Temperature anomaly relative to the 1951-1980 average")
plt.legend();

In [ ]:
df_glob_anomaly = pd.DataFrame(df_glob_anomaly.dropna())
df_glob_anomaly.head(10)

***

## Gridded Data Set

Resources: [Overview of current atmospheric reanalyses](http://reanalyses.org/atmosphere/overview-current-atmospheric-reanalyses)

### ECMWF CERA-20C: 1901-2010

[What is CERA-20C](https://software.ecmwf.int/wiki/display/CKB/What+is+CERA-20C)

[Monthly Means of Daily Mean](https://www.ecmwf.int/en/research/climate-reanalysis/cera-20c)

Stream: Ensemble data assimilation monthly means of daily means  
Area: 73.5°N 27.0°W 33.0°N 45.0°E  
Type: Analysis  
Number: 0 to 9  
Version: 1  
Grid: 1.0° x 1.0°   
Type of level: Surface  
Parameter: 2 metre temperature  
Class: ERA-CLIM2 coupled reanalysis of the 20th-century (CERA-20C) 

### Load data set

In [ ]:
FILENAME = "_grib2netcdf-atls06-95e2cf679cd58ee9b4db4dd119a05a8d-b0bnA_.nc"
ds_cera = xr.open_dataset('../data/'+FILENAME)

In [ ]:
ds_cera

In [ ]:
df_cera = ds_cera.to_dataframe()
print(df_cera.shape)
df_cera.sample(20)

### Compute a single value for a specific meteorological parameter, by avagering all 10 ensemble members

In [ ]:
df_cera = ds_cera.mean(dim='number')
df_cera

### Compute temperature in °C

In [ ]:
print(df_cera.t2m.shape)
df_cera.t2m

In [ ]:
df_cera['t2m'] = df_cera.t2m - 273.15
df_cera

In [ ]:
df_cera.t2m.plot();

In [ ]:
df_cera.t2m[0].plot();

#### Nicer plotting using cartopy

_Feel free to experiment with different map projections provided by the cartopy library (see [here](http://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html) for a list of featured projections)_

In [ ]:
def cuteplot(ds, var, idx=0): 
    '''
    Function to plot xarray data set on different map projections
    '''
    ax = plt.axes(projection=ccrs.Mollweide())    
    ds[var][idx].plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(),
                                x='longitude', y='latitude', add_colorbar=True)
    ax.coastlines(resolution='50m')
    ax.gridlines()
    plt.tight_layout();
    
cuteplot(df_cera, 't2m', idx=0)

### Compute average for the period 1951-1980

In [ ]:
df_cera_reference_period = df_cera.sel(time=slice('1951', '1980'))
df_cera_reference_period

In [ ]:
df_cera_reference_period_mean = df_cera_reference_period.mean(dim='time')
df_cera_reference_period_mean

In [ ]:
df_cera_reference_period_mean.t2m.plot();

### Compute temperature anomaly for 10 year rolling window

In [ ]:
df_cera_anomaly = df_cera.copy()
df_cera_anomaly['t2m'] = df_cera.t2m - df_cera_reference_period_mean.t2m

In [ ]:
w # 10 years on monthly scale

In [ ]:
df_cera_anomaly_10y = df_cera_anomaly.rolling(time=w, center=True).mean()
df_cera_anomaly_10y 

In [ ]:
#df_cera_anomaly_10y.t2m[60].plot();
cuteplot(df_cera_anomaly_10y,'t2m',idx=60)

### Find coordinates of Telegrafenberg, Potsdam

In [ ]:
import geopandas as gpd

In [ ]:
pik = gpd.tools.geocode('Telegrafenberg 14473 Potsdam')
pik

In [ ]:
X = pik.geometry.x.values[0]
Y = pik.geometry.y.values[0]
print(X,Y)

#### Sanity check of coordinates

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[Y, X], zoom_start=5, min_zoom=2)
folium.Marker([Y, X], popup='GFZ').add_to(m)
folium.TileLayer('stamentoner', min_zoom=2).add_to(m)
folium.LayerControl().add_to(m)
m

### Use PIKcoordinates to extract data from CERA dataset

In [ ]:
df_cera_anomaly_10y

In [ ]:
print(X,Y)

In [ ]:
gfz_anomaly = df_cera_anomaly_10y.sel(latitude=[Y], longitude=[X], method='Nearest')
gfz_anomaly

In [ ]:
gfz_anomaly.t2m.plot();

### Continue with pandas ...

In [ ]:
df_gfz_anomaly = gfz_anomaly.to_dataframe()
df_gfz_anomaly.head(10)

In [ ]:
# Clean up
df_gfz_anomaly = (df_gfz_anomaly.reset_index().
                  drop(['longitude', 'latitude'], axis=1).
                  set_index('time').dropna())
df_gfz_anomaly.index = df_gfz_anomaly.index.to_period('M')
df_gfz_anomaly.sample(10)

## Combine data sets

**Recall**  
* Global temperatur anomaly data set: `df_glob_anomaly`   
* Potsdam temperatur anomaly data set: `df_gfz_anomaly`

### Merge data on index

In [ ]:
df_glob_anomaly.index

In [ ]:
df_gfz_anomaly.index

In [ ]:
df_gfz_anomaly['glob_anomaly'] = df_glob_anomaly
df_gfz_anomaly.head()

### Final Plot

In [ ]:
df_gfz_anomaly.plot()
plt.ylabel('Temperature anomaly')
plt.title('Global and Local Temperature Anomaly relative \nto the 1951-1980 Average (10-Year Rolling Window)', 
          size=18)
plt.legend(['Local Scale (ECMWF CERA-20C)',
            'Global Scale (Berkely Earth Data Set)']);

***
**Conclusion:** The analysis of the gridded data set ECMWF CERA-20C, indicates that the earth surface temperature rise at Potsdam, Germany is in accordance with the global trend. 